In [1]:
from nsepy import get_history
import datetime
from datetime import date
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from nsepy.derivatives import get_expiry_date

Table = pd.read_csv('Optionpivot.csv')
Table['month'] = pd.to_datetime(Table['Date']).dt.strftime('%m')

Table.head()

,Date,Nifty,Type,Expiry,9000,9100,9200,9300,9400,9500,...,11900,12000,12100,12200,12300,12400,12500,12600,12700,month
0,2018-04-02,10264.0,CE,2018-04-26,1249.95,1158.00,1051.10,956.05,857.95,763.25,...,0.70,0.75,0.65,NaN,NaN,NaN,NaN,NaN,NaN,04
1,2018-04-02,10264.0,CE,2018-05-31,NaN,1463.05,1020.40,1273.15,1180.05,806.85,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,04
2,2018-04-02,10264.0,CE,2018-06-28,NaN,NaN,994.90,1228.95,1415.90,780.00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,04
3,2018-04-02,10264.0,PE,2018-04-26,3.70,5.00,6.55,8.05,11.10,14.50,...,1700.45,1725.50,848.30,NaN,NaN,NaN,NaN,NaN,NaN,04
4,2018-04-02,10264.0,PE,2018-05-31,NaN,20.85,26.75,31.70,39.90,49.70,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,04


In [2]:
Table1 = Table.groupby('month').first()
Table1['M'] = pd.to_datetime(Table1['Date']).dt.strftime('%m')
Table1['ATM_Strike'] = round(Table1['Nifty']/100)*100

Table3 = pd.DataFrame({"Date":['0'], "Close":['0'], "Expiry":['0'], 
                       "Type":['0'], "month":['0'], "ATM":['0'], 
                       "OTM":['0'], "Ratio":['0'], "P&L":['0']})
Table4 = pd.DataFrame({"Date":['0'], "Close":['0'], "Expiry":['0'], 
                       "Type":['0'], "month":['0'], "ATM":['0'], 
                       "OTM":['0'], "Ratio":['0'], "P&L":[0]})
for x in range (0,9):
    e=Table1.M[x]
    d=Table1.Expiry[x]
    a= int((Table1.ATM_Strike[x]))
    a=a+200
    b = a+200
    f1 = str(a)
    f2 = str(b)
    Table2 = pd.DataFrame({"Date": Table["Date"], "Close": Table["Nifty"], 
                           "Expiry": Table["Expiry"], "Type": Table["Type"], 
                       "month": Table["month"], "ATM": Table[f1], "OTM": Table[f2]})
    Table2= Table2.where((Table2.month == e) & (Table2.Expiry == d) & (Table2.Type == "CE"))
    Table2 = Table2.dropna()
    Table2['Ratio']=Table2['ATM']-(Table2['OTM']*2)
    Table2['P&L']=Table2['Ratio'].diff().cumsum()
    Table3 = Table3.append(Table2)
    Table4= Table4.append(Table2.where((Table2.Date == d)))
    Table4 = Table4.dropna()
Table4.drop(["Date", "Close", "Expiry", "Type", "ATM", "OTM", "Ratio"], axis = 1, inplace = True)
Table4['Total_P&L']=Table4['P&L'].cumsum()
print (Table4)

    month     P&L  Total_P&L
0       0    0.00       0.00
108    04   78.35      78.35
252    05  -27.55      50.80
372    06  -20.30      30.50
492    07  130.10     160.60
630    08   50.30     210.90
738    09  -24.35     186.55
846    10  -18.20     168.35
940    11  129.40     297.75
980    12  -17.80     279.95


In [20]:
Table3.to_csv("Strategy_computation.csv", index=True, encoding='utf8')